In [2]:
import sys

parent_dir = 'Predict-Future-Sales'
p_sub = sys.path[0]

ride = ''
for path in p_sub.split('/'):
    if path != parent_dir:
        ride = ride + path + '/'
    else:
        ride = ride + path + '/'
        break
sys.path[0] = ride

In [3]:
from typing import Tuple

import numpy as np
import pandas as pd
import torch

from module.lino_module.preprocess import _mode_of_freq

In [4]:
import pickle
import pathlib
import glob

model_path = './models/'
p_file = pathlib.Path(model_path)
file_name = [i.name for i in p_file.iterdir()]

with open (model_path + file_name[0], 'rb') as f:
    obj = pickle.load(f)

In [5]:
def _tde_for_inference(ds: pd.Series, seq, d_model, dilation) -> np.ndarray:
    for_array = []
    for i in range(d_model):
        if i != 0:
            for_array.append(ds[-seq - i * dilation: -i * dilation])
        else:
            for_array.append(ds[-seq:])
    time_delay_embedded = np.array([content for content in reversed(for_array)])

    return time_delay_embedded

def _src_tgt_split(tded: np.ndarray,
                   src_seq: int,
                   tgt_seq: int) -> Tuple[np.ndarray]:
    """エンコーダ入力とデコーダ入力への分割"""
    src = tded[:, :src_seq]
    tgt = tded[:, -tgt_seq:]
    return src, tgt

In [6]:
model = obj['model']
env_kwrgs = obj['env']

def _inference(model, data, seq, d_model, dilation, src_tgt_seq):
    src_seq, tgt_seq = src_tgt_seq
    df = _mode_of_freq(data).item_cnt_day.values
    df = tde_for_inference(df, seq, d_model, dilation)
    src, tgt = _src_tgt_split(df, src_seq, tgt_seq)

    src = torch.from_numpy(src.astype(np.float32)).T.unsqueeze(0)
    tgt = torch.from_numpy(tgt.astype(np.float32)).T.unsqueeze(0)
    model.eval()
    return model(src, tgt)


In [7]:
inference(model, **env_kwrgs)

tensor([[[1.7198],
         [1.4385]]], grad_fn=<ViewBackward0>)